In [3]:
from keras.preprocessing.image import ImageDataGenerator


In [4]:
# 이미지 제너레이터를 정의합니다.
train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

In [7]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [8]:
class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes, finalAct="softmax"):
		# 인풋 이미지의 차원과, 채널에 해당하는 축을 설정하여 모델을 초기화합니다
		# "channels_last"는 채널의 축이 마지막에 오는 것을 의미합니다
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
 
 		# 만약 "channels_first"를 사용한다면, 인풋 이미지의 차원을
		# 그에 맞게 바꿔줍니다
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

    # CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))
  

    # (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
 
		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
  

    # FC => RELU
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
 
 		# 단일 라벨 분류는 *softmax* 활성화 함수를 사용합니다
		# 다중 라벨 분류는 *sigmoid* 활성화 함수를 사용합니다
		model.add(Dense(classes))
		model.add(Activation(finalAct))
 
 		# 네트워크 아키텍처를 반환합니다
		return model

In [11]:
import matplotlib
matplotlib.use("Agg")
 
# 필요한 패키지들을 가져옵니다
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
# from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [6]:
# Make Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../../',
    x_col = 'image',
    y_col = class_col,
    target_size = (96, 96),
    color_mode='rgb',
    class_mode='other',
    batch_size=batch_size,
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../../',
    x_col = 'image',
    y_col = class_col,
    target_size = (96, 96),
    color_mode='rgb',
    class_mode='other',
    batch_size=batch_size,
    shuffle=True
)

NameError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -qq "/content/drive/MyDrive/이게뭐약/AI/색깔모양분류/all/Allcolorimg.zip"

In [17]:
# import shutil

# for filename in os.listdir('/content/colorimg4'):
#   if os.path.isfile('/content/colorimg/'+filename): continue
#   else : shutil.move('/content/colorimg4/'+filename,'/content/colorimg/')

# !zip -r /content/drive/MyDrive/이게뭐약/AI/색깔모양분류/all/Allcolorimg.zip ./colorimg/

In [4]:
import os
len(os.listdir('/content/colorimg'))

19966

In [ ]:
os.listdir('/content/colorimg')

In [ ]:
# # construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True, help="path to input dataset (i.e., directory of images)")
# ap.add_argument("-m", "--model", required=True, help="path to output model")
# ap.add_argument("-l", "--labelbin", required=True, help="path to output label binarizer")
# ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output accuracy/loss plot")
# args = vars(ap.parse_args())

In [14]:
import easydict
 
args = easydict.EasyDict({
        "dataset": True,
        "model": True,
        "labelbin": True,
        "plot": "plot.jpg"

})

In [15]:
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [16]:
# 이미지 경로를 섞어줍니다
print("[INFO] loading images...")
imagePaths = "/content/colorimg"
# random.seed(42)
# random.shuffle(imagePaths)
 


[INFO] loading images...


In [17]:
file_list = os.listdir(imagePaths)

In [ ]:
file_list

In [18]:
file_list[9].split("-")[0].split("_")

['장방형', '파랑']

In [19]:
# 데이터와 라벨을 초기화합니다
data = []
labels = []

In [20]:
# 인풋 이미지들에 대해 아래의 반복문을 수행합니다
for imagePath in file_list:
  image = cv2.imread('/content/colorimg/'+imagePath)
  # print(image)
  if image is not None : 
    image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
 
    # # # 이미지 경로에서 라벨을 추출한 후, 라벨 리스트를 업데이트합니다
    l = label = imagePath.split("-")[0].split("_")
    labels.append(l)

In [21]:
len(data)

19961

In [22]:
labels[:10]

[['원형', '하양'],
 ['타원형', '주황'],
 ['육각형', '파랑'],
 ['장방형', '초록'],
 ['장방형', '하양'],
 ['타원형', '갈색'],
 ['장방형', '갈색'],
 ['타원형', '주황'],
 ['타원형', '하양'],
 ['장방형', '파랑']]

In [23]:
data[:5]

[array([[[236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 166.],
         ...,
         [236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 166.]],
 
        [[236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 166.],
         ...,
         [236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 166.]],
 
        [[236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 166.],
         ...,
         [236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 166.]],
 
        ...,
 
        [[235., 193., 164.],
         [236., 193., 166.],
         [236., 193., 166.],
         ...,
         [237., 192., 165.],
         [236., 193., 166.],
         [235., 193., 165.]],
 
        [[236., 193., 164.],
         [236., 193., 166.],
         [236., 193., 166.],
         ...,
         [236., 193., 166.],
         [236., 193., 166.],
         [236., 193., 165.]],
 
        [[235., 193., 164.],
 

In [24]:
import tensorflow as tf
print(tf.__version__)

tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

2.4.1


True

In [25]:
# 모든 픽셀 값이 [0, 1]의 범위 내에 오도록 변환합니다 
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
# print("[INFO] data matrix: {} images ({:.2f}MB)".format(len(file_list), data.nbytes / (1024 * 1000.0)))

In [26]:
# scikit-learn의 다중 라벨 이진화 함수를 사용해 라벨을 이진화 합니다
print("[INFO] class labels:")
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
 
# 나올 수 있는 모든 라벨들을 출력합니다
for (i, label) in enumerate(mlb.classes_):
  print("{}. {}".format(i + 1, label))

[INFO] class labels:
1. 갈색
2. 갈색, 진한
3. 갈색, 투명
4. 검정
5. 검정, 투명
6. 기타
7. 남색
8. 남색, 투명
9. 노랑
10. 노랑, 투명
11. 마름모형
12. 반원형
13. 보라
14. 보라, 투명
15. 분홍
16. 분홍, 옅은
17. 분홍, 진한
18. 분홍, 투명
19. 빨강
20. 빨강, 투명
21. 사각형
22. 삼각형
23. 연두
24. 연두, 투명
25. 오각형
26. 원형
27. 육각형
28. 자주
29. 자주, 투명
30. 장방형
31. 주황
32. 주황, 투명
33. 청록
34. 청록, 투명
35. 초록
36. 초록, 투명
37. 타원형
38. 투명
39. 파랑
40. 파랑, 옅은
41. 파랑, 투명
42. 팔각형
43. 하양
44. 하양, 갈색
45. 하양, 노랑
46. 하양, 빨강
47. 하양, 주황, 투명
48. 하양, 초록
49. 하양, 투명
50. 하양, 파랑
51. 회색


In [27]:
mlb.transform([("원형", "노랑")])

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]])

In [28]:
# 데이터의 80%를 학습에, 나머지 20%를 테스트에 사용하기 위해
# 데이터를 나누는 과정입니다
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.2, random_state=42)
 
# 이미지 오그멘테이션을 위한 제너레이터를 초기화합니다
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
data = []
labels = []

In [30]:
# 다중 라벨 분류를 수행할 수 있도록 sigmoid 활성화 함수를
# 네트워크의 마지막 레이어로 설정합니다
print("[INFO] compiling model...")
model = SmallerVGGNet.build(
	width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(mlb.classes_),
	finalAct="sigmoid")

# 옵티마이저를 초기화합니다
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

[INFO] compiling model...


In [ ]:
# 각각의 결과 라벨을 독립적인 베르누이 분포로 취급하기 위해
# 범주형 교차 엔트로피 대신 이진 교차 엔트로피를 사용하여 모델을 컴파일합니다
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# 네트워크를 학습시킵니다
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

[INFO] training network...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/75
499/499 [==============================] - 70s 74ms/step - loss: 0.3622 - accuracy: 0.1715 - val_loss: 0.1759 - val_accuracy: 0.2039
Epoch 2/75
499/499 [==============================] - 37s 73ms/step - loss: 0.0538 - accuracy: 0.3061 - val_loss: 0.0888 - val_accuracy: 0.4648
Epoch 3/75
499/499 [==============================] - 36s 73ms/step - loss: 0.0449 - accuracy: 0.3149 - val_loss: 0.0489 - val_accuracy: 0.2549
Epoch 4/75
499/499 [==============================] - 36s 73ms/step - loss: 0.0425 - accuracy: 0.3238 - val_loss: 0.0480 - val_accuracy: 0.4786
Epoch 5/75
499/499 [==============================] - 36s 73ms/step - loss: 0.0402 - accuracy: 0.3227 - val_loss: 0.1043 - val_accuracy: 0.3794
Epoch 6/75
499/499 [==============================] - 37s 74ms/step - loss: 0.0385 - accuracy: 0.3275 - val_loss: 0.0873 - val_accuracy: 0.2612
Epoch 7/75
499/499 [==============================] - 37s 73ms/step - loss: 0.0379 - accuracy: 0.3189 - val_loss: 0.0973 - val_accuracy:

In [ ]:
# 모델을 디스크에 저장합니다
from keras.models import load_model

model.save('multilabel_model.h5')

# print("[INFO] serializing network...")
# model.save(args["model"])
 
# `MultiLabelBinarizer`를 디스크에 저장합니다
print("[INFO] serializing label binarizer...")
f = open('/content/labelbin.txt', "wb")
f.write(pickle.dumps(mlb))
f.close()

In [ ]:
H.history

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
# 학습 로스와 정확도를 그래프로 그려줍니다
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.show()
plt.savefig(args["plot"])

In [ ]:
# loss, acc curve

acc = H.history['accuracy']
val_acc = H.history['val_accuracy']

loss = H.history['loss']
val_loss = H.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Valid Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Valid Loss')
plt.show()

새로운 이미지에 적용

In [ ]:
# 필요한 패키지들을 가져옵니다
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from google.colab.patches import cv2_imshow
from PIL import ImageFont, ImageDraw, Image

In [ ]:
def test(img_dir):
  
  # 이미지를 로드합니다
  image = cv2.imread(img_dir)
  output = imutils.resize(image, width=400)
  
  # # 분류를 위한 이미지 전처리를 수행합니다
  image = cv2.resize(image, (96, 96))
  image = image.astype("float") / 255.0
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0)
  # # 학습된 네트워크와 `MultiLabelBinarizer`를 로드합니다
  print("[INFO] loading network...")
  model = load_model('/content/multilabel_model.h5')
  mlb = pickle.loads(open('/content/labelbin.txt', "rb").read())

  # # 이미지에 대한 분류를 수행한 후, 
  # # 확률이 가장 높은 두 개의 클래스 라벨을 찾습니다
  print("[INFO] classifying image...")
  proba = model.predict(image)[0]
  idxs = np.argsort(proba)[::-1][:2]

  # # 확률이 높은 라벨들에 대해 아래의 반복문을 수행합니다

  for (i, j) in enumerate(idxs):
    label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
    cv2.putText(output, label, (10, (i * 30) + 25), 
      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

  # 각 라벨에 대한 확률을 출력합니다
  for (label, p) in zip(mlb.classes_, proba):
    print("{}: {:.2f}%".format(label, p * 100))
 
  # 결과 이미지를 출력합니다
  cv2_imshow(output)
  cv2.waitKey(0)


In [ ]:
test('/content/test/out5.jpg')

In [ ]:
testpath = '/content/test'

In [ ]:
test_list = os.listdir(testpath)
test_list

In [ ]:
#png 파일 있으면 jpg 로 변환해주기

for sampleimg in test_list :
  if os.path.splitext(sampleimg)[-1] != '.jpg':
    im = Image.open(testpath + '/'+sampleimg)
    im = im.convert("RGB")
    im.save(testpath + '/'+'%s.jpg'%os.path.splitext(sampleimg)[0])
    print(sampleimg)
    os.remove(testpath + '/'+sampleimg)

